In [1]:
#インポート
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import openpyxl

In [2]:
#ローカルファイルの読み込み
import data_make as dm   #データを取得するためのファイル
import Input_form as Info #ファイル名の入力などを行うためのファイル

In [3]:
#データファイルの選択
#local_ripository_path = Info.folder_select()
#dot_name = Info.file_select('C:/Users/jackp/Research/spatical_scan/data/','dot','ファイルの依存関係(dot形式)')

#リポジトリのURL指定
repo_url = Info.get_text('コミット情報を取得するのリポジトリのURL')
from_ver = Info.get_text('変更前のブランチ')
to_ver = Info.get_text('変更後のブランチ')

#APIのid, KEY
client_id = Info.get_text('クライアントID')
client_secret = Info.get_text('クライアントシークレット')

In [4]:
print('リポジトリのULR : ' + repo_url + '\n'
      #'行数データのファイル : ' + local_ripository_path + '\n'
      #'依存関係のファイル : ' + dot_name + '\n'
      'クライアントID : ' + client_id + '\n'
      'クライアントシークレット' + client_secret
     )

リポジトリのULR : https://github.com/apache/tomcat.git
クライアントID : ed610b1e022dbc51c7f8
クライアントシークレット6d9aac220c7af89abef64524bd15c25d1d8855e5


In [9]:
#ファイルの行数とファイルの依存関係の取得
java_line, java_module = dm.get_java_line(repo_url, from_ver, to_ver, client_id, client_secret)
#コミット情報の取得
java_commit = dm.get_java_commit(repo_url, from_ver, to_ver, client_id, client_secret,java_line)
#変更行数の割合を取得
prob_list = dm.get_prob_list(java_line, java_commit)

api : https://api.github.com/repos/apache/tomcat/git/trees/8.5.43


KeyError: 'tree'

In [10]:
#結果の出力
print('java_module')
print(len(java_module))
print('java_line')
print(len(java_line))
print('java_commit')
print(len(java_commit))
print('prob_list')
print(len(prob_list))

java_module


NameError: name 'java_module' is not defined

In [ ]:
#依存関係にないファイルを除外
j_commit = []
j_line = []
p_list  = []

for i in tqdm(java_module):
    for j in java_commit:
        if (j[0] in i) and (j not in j_commit):
            j_commit.append(j)

for i in tqdm(java_module):
    for j in java_line:
        if (j[0] in i) and (j not in j_line):
            j_line.append(j)

for i in tqdm(java_module):
    for j in prob_list:
        if (j[0] in i) and (j  not in p_list):
            p_list.append(j)
            
java_commit = sorted(j_commit, key=lambda x:x[1], reverse=True)
java_line = sorted(j_line, key=lambda x:x[1], reverse=True)
prob_list  = sorted(p_list, key=lambda x:x[1], reverse=True)

In [ ]:
#結果の出力
print('java_module')
print(len(java_module))
print('java_line')
print(len(java_line))
print('java_commit')
print(len(java_commit))
print('prob_list')
print(len(prob_list))

In [ ]:
#空間スキャン検定に必要なファイルの読み込み
import CircularScan as cs

In [ ]:
#CSのタイプを選択
#scan_type = 'hop'  #'hop'ならC_scan_hops()を使用.
scan_type = 'default' #それ以外ならC_scan()を使用する

#C_scan_hops()の設定
iter_num = 1     #CSの反復回数
hop_count = 2     #C_scan_hops()を使うときのホップ数

#C_scanの設定
threshold = 0.25   #領域拡大の停止条件(threshold=0.05 → 領域の行数が全体の5%以上になったら停止)

#実行
result = cs.CS_run(java_line, java_commit, java_module, prob_list, scan_type, iter_num, hop_count, threshold)

In [ ]:
result

In [ ]:
#保存ファイル名の決定
Excel_name = 'result'

#データフレームへの変換
df = pd.DataFrame(result, columns=['ファイル名','p値'])

#エクセルデータとして結果を保存
df.to_excel('../result/'+Excel_name+'.xlsx', sheet_name='new_sheet_name')